In [1]:
import pandas as pd
from sqlalchemy import create_engine
df = pd.read_csv(r"C:\Users\FUT_Nandhini\Downloads\Sample_data.csv")
engine = create_engine("sqlite:///My_db.db",echo=True)
df.to_sql('your_table',con=engine,if_exists = 'replace',index=False)



2025-02-15 23:38:57,714 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-15 23:38:57,714 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("your_table")
2025-02-15 23:38:57,714 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-15 23:38:57,714 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("your_table")
2025-02-15 23:38:57,722 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-15 23:38:57,723 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-02-15 23:38:57,724 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-15 23:38:57,724 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-02-15 23:38:57,725 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-15 23:38:57,727 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("your_table")
2025-02-15 23:38:57,728 INFO sqlalchemy.engine.Engine [raw sql] ()
202

500

In [46]:
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_pretrained("defog/sqlcoder-7b-2")

e:\texttosql\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\FUT_Nandhini\.cache\huggingface\hub\models--defog--sqlcoder-7b-2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [49]:
#from transformers import SQLCoderTokenizer, SQLCoderModel
#from transformers import SQLCoderTokenizer, SQLCoderModel
#from sqlcoder import SQLCoderTokenizer, SQLCoderModel

import sqlalchemy
import pandas as pd

# 1. Database connection (using SQLAlchemy) - Assuming you've already done this
engine = sqlalchemy.create_engine("sqlite:///My_db.db", echo=True)

# 2. Load SQLCoder tokenizer and model
#tokenizer = SQLCoderTokenizer.from_pretrained('defog/sqlcoder')
model = tokenizer.from_pretrained('defog/sqlcoder-7b-2')

# 3. Get the database schema
def get_table_schemas(engine):
    """Extracts table schemas from the SQLAlchemy engine."""
    inspector = sqlalchemy.inspect(engine)
    schemas = {}
    for table_name in inspector.get_table_names():
        columns = inspector.get_columns(table_name)
        column_definitions = []
        for column in columns:
            column_name = column['name']
            column_type = str(column['type'])  # Convert type to string
            column_definitions.append(f"{column_name} {column_type}")
        schemas[table_name] = f"CREATE TABLE {table_name} ({', '.join(column_definitions)});"
    return "\n".join(schemas.values())

db_schema = get_table_schemas(engine)

# 4. Define your natural language question
question = "What is the average price of products?"

# 5. Construct the prompt for SQLCoder (critical step)
prompt = f"""
### Instructions:
Your task is to convert a question into a SQL query, given a database schema.
Adhere to these rules:
- Deliberately go through the question and database schema word by word to appropriately answer the question
- Use Table Aliases to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.
- When creating a ratio, always cast the numerator as float
### Input:
Generate a SQL query that answers the question `{question}`.
This query will run on a database whose schema is represented in this string:
{db_schema}
### Response:
Based on your instructions, here is the SQL query I have generated to answer the question `{question}`:
"""


2025-02-15 23:29:30,537 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-15 23:29:30,537 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-02-15 23:29:30,551 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-15 23:29:30,554 INFO sqlalchemy.engine.Engine ROLLBACK
2025-02-15 23:29:30,556 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-15 23:29:30,557 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("your_table")
2025-02-15 23:29:30,559 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-15 23:29:30,560 INFO sqlalchemy.engine.Engine ROLLBACK


In [50]:

# Now you can use the tokenizer and model to generate the SQL query
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs)
sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(sql_query)

TypeError: 'tokenizers.Tokenizer' object is not callable

In [56]:
import pandas as pd
from sqlalchemy import create_engine
from transformers import AutoTokenizer, AutoModelForCausalLM
import sqlalchemy

# Load your CSV data into a DataFrame
df = pd.read_csv(r"C:\Users\FUT_Nandhini\Downloads\Sample_data.csv")

# Create a SQLite engine
engine = create_engine("sqlite:///My_db.db", echo=True)

# Write the DataFrame to a SQL table
df.to_sql('sample_data', con=engine, if_exists='replace', index=False)

# Load SQLCoder tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('defog/sqlcoder-7b-2')
model = AutoModelForCausalLM.from_pretrained('defog/sqlcoder-7b-2')

# Function to get the database schema
def get_table_schemas(engine):
    """Extracts table schemas from the SQLAlchemy engine."""
    inspector = sqlalchemy.inspect(engine)
    schemas = {}
    for table_name in inspector.get_table_names():
        columns = inspector.get_columns(table_name)
        column_definitions = []
        for column in columns:
            column_name = column['name']
            column_type = str(column['type'])  # Convert type to string
            column_definitions.append(f"{column_name} {column_type}")
        schemas[table_name] = f"CREATE TABLE {table_name} ({', '.join(column_definitions)});"
    return "\n".join(schemas.values())

# Get the database schema
db_schema = get_table_schemas(engine)
print(db_schema)



# Construct the prompt for SQLCoder
prompt = f"""
### Instructions:
Your task is to convert a question into a SQL query, given a database schema.
Adhere to these rules:
- Deliberately go through the question and database schema word by word to appropriately answer the question
- Use Table Aliases to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.
- When creating a ratio, always cast the numerator as float
### Input:
Generate a SQL query that answers the question `{question}`.
print the columns names and values completely as asked in the question
Very important note: the output should be consize and complete
This query will run on a database whose schema is represented in this string:
{db_schema}
### Response:
Based on your instructions, here is the SQL query I have generated to answer the question `{question}`:
### Response:
Based on your instructions, here is the SQL query I have generated to answer the question `{question}`:
"""




2025-02-16 21:58:09,192 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-16 21:58:09,200 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sample_data")
2025-02-16 21:58:09,200 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-16 21:58:09,204 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sample_data")
2025-02-16 21:58:09,206 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-16 21:58:09,208 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-02-16 21:58:09,208 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-16 21:58:09,210 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-02-16 21:58:09,210 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-16 21:58:09,212 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("sample_data")
2025-02-16 21:58:09,213 INFO sqlalchemy.engine.Engine [raw sql] ()


Loading checkpoint shards: 100%|██████████| 3/3 [00:08<00:00,  2.72s/it]


2025-02-16 21:58:19,928 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-16 21:58:19,928 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-02-16 21:58:19,928 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-16 21:58:19,935 INFO sqlalchemy.engine.Engine ROLLBACK
2025-02-16 21:58:19,935 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-16 21:58:19,935 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("sample_data")
2025-02-16 21:58:19,940 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-16 21:58:19,943 INFO sqlalchemy.engine.Engine ROLLBACK
2025-02-16 21:58:19,946 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-16 21:58:19,946 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("your_table")
2025-02-16 21:58:19,950 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-16 21:58:19,953 INFO sqlalchemy.engine.Engine ROLLBACK
CREATE TABLE sample_data (Unnamed: 0 BIGINT, Job Card

In [57]:
question = "Get me top 25 aggregate by warranty cost since april 2023"


In [58]:
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Generate SQL query with increased max_new_tokens
output = model.generate(input_ids, max_new_tokens=150, temperature=0.5)  # Adjust max_new_tokens as needed

# Decode and print the generated SQL query
generated_sql = tokenizer.decode(output[0], skip_special_tokens=True)

# Remove prompt from output (if necessary)
generated_sql_cleaned = generated_sql[len(prompt):].strip()

print("Generated SQL Query:", generated_sql_cleaned)

e:\texttosql\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated SQL Query: SELECT s.Model Name, SUM(s.Total Warranty Cost) AS total_warranty_cost FROM sample_data s WHERE s.Job Card Date >= '2023-04-01' GROUP BY s.Model Name ORDER BY total_warranty_cost DESC LIMIT 25;


In [56]:
import torch
print(torch.__version__)

2.6.0+cpu
